# Algorithms used for the thesis. 

In this jupyter notebook I saved the algorithms used for my thesis. 
In this file is presented only the code used. For a better explaination of the algorithms, results and problem related to the implementation the reader is invited to look at the third chapter of my thesis.

In [ ]:
import numpy as np
import math
import random
import scipy.spatial as sp
from matplotlib import pyplot as plt



## Simplex Projection



To calculate the volume of a simplex I used the algorithm found [here](https://mathworld.wolfram.com/Cayley-MengerDeterminant.html), which uses the Cayley-Menger determinant.

To check if the point is contained in a simplex I used an algorithm that uses the barycentric coordinates found [here](https://math.stackexchange.com/questions/1226707/how-to-check-if-point-x-in-mathbbrn-is-in-a-n-simplex).

In [ ]:
from itertools import combinations 
from scipy.spatial import distance_matrix

def nCr(n,r):
    """class that given 2 positive integer, return the number of combinations"""
    f = math.factorial
    return f(n) / f(r) / f(n-r)

def cayley_menger_determinant(matrix,E_dim):
    """cayley menger determinant: hypervolume of an irregular simplex:\n
        matrix - irregular simplex
        E_dim - dimension of the irregular simplex
    """
    dist_matrix = np.zeros((E_dim+1,E_dim+1))
    for i in range(E_dim+1):
        for j in range(E_dim+1):
            dist_matrix[i][j]=np.linalg.norm(matrix[i]-matrix[j])**2
    addc=np.ones(E_dim+1)
    addr=np.ones(E_dim+1)
    addr=np.insert(addc, 0, 0)
    B=np.column_stack((addc,dist_matrix))
    B=np.vstack((addr,B))

    vol=abs( np.linalg.det(B)/( 2**E_dim * math.factorial(E_dim)**2 ) )
    vol=math.sqrt(vol)
    return vol


class Simplex_Projection:
    """Sugihara_and_May_algorithm class"""

    def __init__(self, matrix, N, E_dim, m_bool):
        """
        SP class constructor.\n
        matrix (N-E+1,E) - whose rows are points in embedding dimension;\n
        N - longness of time serie;\n
        E_dim - dimension of each point;
        m_bool - if m_bool=0 means that the matrix need to be built 
                 overlapping the time series.
                 if m_bool=1 the matrix need to be built not overlapping the time serie
        accepted - int that counts the number of times "prediction's method" finds a simplex for a point
        attempted - int that counts the number of times "prediction's method" is called
        in - boolean variable if is true the point is in the simplex, if it is false is out of the simplex
        """
        self.matrix = matrix
        self.N = N
        self.E_dim = E_dim
        self.bool = m_bool
        self.accepted=0
        self.attempted=0
        self.inside=0
        
    def get_acceptance(self):
        print(f'self.attempted: {self.attempted}')
        print(f'self.accepted: {self.accepted}')
        if(self.attempted==0):
            return 0
        return self.accepted/self.attempted
        
    def check_inside_simplex(self,Vector,matrix):
        """ method used to verify if a point 'Vector' is inside a simplex 'matrix' """
        assert matrix[0].size == self.E_dim and matrix[:,0].size == self.E_dim + 1
        
        #Building the cm-basis of R^E of our simplex
        basis_cm = np.zeros((self.E_dim,self.E_dim))
        for i in range(self.E_dim):
            for j in range(self.E_dim):
                basis_cm[i][j] = matrix[i+1][j] - matrix[0][j]
    
        basis_cm=np.transpose(basis_cm)
        
        #Writing a vector in cm coordinates
        vector_cm = np.zeros(self.E_dim)
        for i in range(self.E_dim):
            vector_cm[i] = Vector[i] - matrix[0][i]
            
        #Solution is the vector of coefficients of our linear system if the rank is maximum, otherwise is the coefficient that minimize the mse    
        solution=np.linalg.lstsq(basis_cm, vector_cm)
        sum_solution=0.0
        cont=0
        for i in range(self.E_dim):
            sum_solution = sum_solution + solution[0][i]
            if(solution[0][i] >= 0):
                cont = cont +1 
        #those are the sufficient and necessary conditions to check if the point is in the simplex        
        if( cont == self.E_dim and sum_solution <= 1):
            return True
        else:
            return False
    
        
    
    def Exponetial_Weighted_Prediction(self, Vector, step):
        """It generates the prediction of some "step" in the future of "Vector" making the exponetial-weighted average of the simplex"""
        assert Vector.size == self.E_dim
        if( self.bool == True):
            assert self.N % self.E_dim == 0
            long = int(self.N/self.E_dim)
            
        else:
            long = self.N - self.E_dim + 1
            
        #Copying the matrix in matrix_shorter
        matrix_shorter = self.matrix
        
        #Deleting the last step_future rows of the matrix because the state space doesn't have their projection in the future
        for i in range(step):
            matrix_shorter = np.delete(matrix_shorter, matrix_shorter[:,0].size-1,axis=0) 
           
        #Calculating the distances between Vector and the points of the state space
        distance = np.zeros(long-step) 
        for i in range (long-step):
            sum=0.0
            for k in range (self.E_dim):
                sum=(Vector[k] - matrix_shorter[i][k])**2
            distance[i] = math.sqrt(sum)
        
        #Arg-sorting the vector of distances
        v_times = np.argsort(distance) 
        #Ordering the vector of distances that will be used to make the estimation of the prediction
        distance = np.sort(distance) 
        
        
        """the next part needs to find the smallest simplex around the point"""
        check = 0
        self.attempted +=1
        while True:
            #Copying v_times in a smaller array
            v_times_check = v_times[0:self.E_dim+1+check]
            #Creating the combinations
            comb = combinations(v_times_check, self.E_dim+1)
            #Number of combinations
            Num_comb=int(nCr(v_times_check.size,self.E_dim+1))
            #Vector which contain the index of the combination k; k=0,...,Num_comb-1
            ordering_combinations = np.arange(Num_comb)
            volume_simplexes = np.empty(0)
            
            iterator_k=0
            number_deletes=0
            
            if ( Num_comb >1500 ):
                #If Num_comb >1500 the algorithm becomes a KNN with K=E
                v_times = v_times[0:self.E_dim]
                self.inside=0
                break    
                
            #I search the simplex in all the E+1 combination of the K nearest neighbors
            for k in list(comb):
                
                #Considering the combination k, and than checking if it contains the point
                v_times_check_current = np.asarray(k)
            
                #Saving the combination in matrix_simplex
                matrix_simplex = np.zeros((self.E_dim+1,self.E_dim))
                for i in range(self.E_dim+1):
                    for j in range(self.E_dim):
                        iterator = int(v_times_check_current[i])
                        matrix_simplex[i][j] = matrix_shorter[iterator][j]
               
                #Checking if Vector is contained in matrix_simplex
                if( self.check_inside_simplex( Vector, matrix_simplex ) ):
                    """if the point is in the simplex i calculate the volume of it"""
                    #Saving the volumes of the simplexes in volume_simplexes
                    vol = cayley_menger_determinant(matrix_simplex,self.E_dim)
                    volume_simplexes=np.append(volume_simplexes,vol)
                else:
                    if( ordering_combinations.size != 0 ):
                        ordering_combinations = np.delete(ordering_combinations,iterator_k-number_deletes)
                        number_deletes+=1
                iterator_k=iterator_k+1
                
            #If it is true, means the algorithm found at least one simplex that contain the point
            if( ordering_combinations.size != 0 ):
                self.accepted +=1
                self.inside=1
                
                #Create a matrix with 2 columns 
                #The first contain the number of the combination, the second the correspective volume
                vol_oredering =  np.column_stack((ordering_combinations,volume_simplexes))
                #Sorting the rows by the volume (i.e. by the second column)
                vol_oredering = vol_oredering[np.argsort(vol_oredering[:, 1])]
               
                #The index of the combination of the smallest volume
                smallest_simplex_combination = vol_oredering[0][0]
                #"contatore" counts the index of the combinations in the next loop "for"
                #At the end of the loop "contatore" is equal to the index of the combination of the smallest simplex
                contatore=0
                #"v_times" saves the time steps of the combination of the simplex
                v_times=np.empty(0)
                comb2 = combinations(v_times_check, self.E_dim+1)
                for j in list(comb2):
                    if( contatore == smallest_simplex_combination ):
                        v_times = np.asarray(j)
                        break
                    contatore = contatore + 1
    
                break
            check=check+1
                    
        """the simplex is built"""
        long_simplex=0
        if(self.inside):
            #If self.inside is True Vector is contained in the simplex
            long_simplex=self.E_dim+1
        else:
            #Otherwise the method becomes a KNN with K=E
            long_simplex=self.E_dim
            
        #Saving the values of the simplex    
        simplex = np.zeros((long_simplex,self.E_dim))
        for i in range (long_simplex):
            for j in range (self.E_dim):
                iteratore=int(v_times[i])
                simplex[i][j]= matrix_shorter[iteratore][j]
 
        #time steps of the new simplex which contains "Vector"
        v_times_prediction = np.zeros(long_simplex)
        for i in range (v_times.size):
            v_times_prediction[i]=v_times[i]+step
         
        #"prediction" is the E-dimensional vector returned by the algorithm
        prediction = np.zeros(self.E_dim)
        #"matrix_prediction" contains the points which form the simplex
        matrix_prediction = np.zeros((long_simplex,self.E_dim))
        for i in range (long_simplex):
            for j in range (self.E_dim):
                iter=int(v_times_prediction[i])
                matrix_prediction[i][j]= self.matrix[iter][j]
          
        #Making the exponential weighted average
        for i in range (self.E_dim):
            sum=0.
            den=0.
            for j in range (long_simplex):
                sum+=matrix_prediction[j][i]*math.exp(-distance[j])
                den+=math.exp(-distance[j])
            sum = sum/den
            prediction[i]=sum
            
        return prediction 






## K-NEAREST NEIGHBORS

In [ ]:
class KNN:
    """KNN algorithm class"""

    def __init__(self, matrix, N, E_dim, K):
        """
        KNN class constructor.\n
        matrix (N-E+1,E) - whose rows are points in embedding dimention;\n
        N - longness of time serie;\n
        E_dim - dimension of each point;
        K - number of neighbors
        """
        self.matrix = matrix
        self.N = N
        self.E_dim = E_dim
        self.K=K
        
        

    def Ave_Prediction(self, Vector, step):
        """It generates prediction of some step in the future making the unweighted-average of the neighbours"""
        assert Vector.size == self.E_dim
            
        #Copying the matrix in matrix_shorter    
        matrix_shorter = self.matrix
        
        #Deleting the last step_future rows of the matrix because the state space doesn't have their projection in the future
        for i in range(step):
            matrix_shorter = np.delete(matrix_shorter, matrix_shorter[:,0].size-1,axis=0)
            
        #Calculating the distances between Vector and the points of the state space
        distance = np.zeros(self.N-self.E_dim+1-step)
        for i in range (self.N-self.E_dim+1-step):
            sum=0.0
            for k in range (self.E_dim):
                sum=(Vector[k] - matrix_shorter[i][k])**2
            distance[i] = math.sqrt(sum)
            
        #Arg-sorting the vector of distances
        v_times = np.argsort(distance)
        #Ordering the vector of distances that will be used to make the estimation of the prediction
        distance = np.sort(distance)
        
        for i in range (self.K,v_times.size):
            #Keeping only the first K points
            v_times = np.delete(v_times,self.K)
            distance = np.delete(distance,self.K)
       
 
        v_times_prediction = np.zeros(self.K)
        for i in range (v_times.size):
            #Time steps of the first K neighbors
            v_times_prediction[i]=v_times[i]+step
         
        
        #Keeping the values of K neigbors in matrix_predictee
        prediction = np.zeros(E)
        matrix_prediction = np.zeros((self.K,E))                
        for i in range (self.K):
            for j in range (self.E_dim):
                iter=int(v_times_prediction[i])
                matrix_prediction[i][j]= self.matrix[iter][j]
                
        #The arithmetical average of the k-nearest neighbors is the estimation of the prediction: the "predictee"   
        for i in range (self.E_dim):
            sum=0
            for j in range (self.K):
                sum+=matrix_prediction[j][i]
            sum = sum/(self.K)
            prediction[i]=sum
            
        return prediction 
    
    
    def Exponetial_Weighted_Prediction(self, Vector, step):
        """It generates prediction of some step in the future making the exponetial-weighted average of the neighbours"""
        assert Vector.size == self.E_dim
        
        #"long" are the number of points in the state space
        long = self.N - self.E_dim + 1
            
        #Copying the matrix in "matrix_shorter"
        matrix_shorter = self.matrix 
        
        #Deleting the last step_future rows of the matrix because the state space doesn't have their projection in the future
        for i in range(step):
            matrix_shorter = np.delete(matrix_shorter, matrix_shorter[:,0].size-1,axis=0) 
            
        #Calculating the distances between Vector and the points of the state space
        distance = np.zeros(long-step) 
        for i in range (long-step):
            sum=0.0
            for k in range (self.E_dim):
                sum=(Vector[k] - matrix_shorter[i][k])**2
            distance[i] = math.sqrt(sum)
        
        #Arg-sorting the vector of distances
        v_times = np.argsort(distance) 
        #Ordering the vector of distances that will be used to make the estimation of the prediction
        distance = np.sort(distance) 
        
        for i in range (self.K,v_times.size):
            #Keeping only the first K points
            v_times = np.delete(v_times,self.K) 
            distance = np.delete(distance,self.K) 
       
 
        v_times_prediction = np.zeros(self.K)
        for i in range (v_times.size):
            #Time steps of the first K neighbors
            v_times_prediction[i]=v_times[i]+step 
         
        #Keeping the values of K neigbors in matrix_predictee
        prediction = np.zeros(self.E_dim)
        matrix_prediction = np.zeros((self.K,self.E_dim)) 
        for i in range (self.K):
            for j in range (self.E_dim):
                iter=int(v_times_prediction[i])
                matrix_prediction[i][j]= self.matrix[iter][j]
         
        #Making the exponential average
        for i in range (self.E_dim): 
            sum=0.
            den=0.
            for j in range (self.K):
                sum+=matrix_prediction[j][i]*math.exp(-distance[j])
                den+=math.exp(-distance[j])
            sum = sum/den
            prediction[i]=sum
            
        return prediction 

    

## THE RECURRENT NEURAL NETWORK

For the implementation of a recurrent neural netwotk (RNN) I followed the tutorial at [this link](https://www.guru99.com/rnn-tutorial.html).

In [3]:
# compose the NN model
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import optimizers, losses, metrics

/Users/lucazilli/miniconda3/envs/calcolo/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lucazilli/miniconda3/envs/calcolo/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lucazilli/miniconda3/envs/calcolo/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/lucazilli/miniconda3/

## NB
check the version of tensorflow. the programme needs the version 1.14.0

In [4]:
print(tf.__version__)

1.14.0


In [ ]:
#hyperparameters
step_futuro=4
n_windows = 100   
n_input =  1
n_output = 1
size_train = 10000+step_futuro
size_test=3000+step_futuro
print(f'size_train {size_train}')
    


In [ ]:
#dataset
train_set = nord_volumes[:size_train]
test_set = nord_volumes[size_train:size_test+size_train]
    
X_batches, y_batches = create_batches(df = train_set,
                                windows = n_windows,
                                input = n_input,
                                output = n_output,
                                step_f=step_futuro,
                                size=size_train)
    
X_test, y_test = create_batches(df = test_set,
                                windows = n_windows,
                                input = n_input,
                                output = n_output,
                                step_f=step_futuro,
                                size=size_test)



In [ ]:
#build the RNN
tf.reset_default_graph()
r_neuron = 1000    

## 1. Construct the tensors
X = tf.placeholder(tf.float32, [None, n_windows, n_input])   
y = tf.placeholder(tf.float32, [None, n_windows, n_output])

## 2. create the model
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=r_neuron, activation=tf.nn.relu)
rnn_output, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)


stacked_rnn_output = tf.reshape(rnn_output, [-1, r_neuron])          
stacked_outputs = tf.layers.dense(stacked_rnn_output, n_output)       
outputs = tf.reshape(stacked_outputs, [-1, n_windows, n_output])   

## 3. Loss + optimization
learning_rate = 0.001  
 
loss = tf.reduce_sum(tf.square(outputs - y))    
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)         
training_op = optimizer.minimize(loss)                                          

init = tf.global_variables_initializer() 
    


In [ ]:
#train
iteration = 150
it_print=10
mse_train_array=[]
mse_test_array=[]
with tf.Session() as sess:
    init.run()
    for iters in range(iteration):
        sess.run(training_op, feed_dict={X: X_batches, y: y_batches})
        if iters % it_print == 0:
            mse_train = loss.eval(feed_dict={X: X_batches, y: y_batches})
            mse_train_array.append(mse_train)
            mse_test = loss.eval(feed_dict={X: X_test, y: y_test})
            mse_test_array.append(mse_test)
            print(iters, "\tMSE E_in:", mse_train)
            print(iters, "\tMSE E_out:", mse_test)
    
    y_pred = sess.run(outputs, feed_dict={X: X_test})
                                      
# y_pred contains the prediction

### Plotting the MSE as function of the iterations

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 22))


plt.plot(np.arange(np.asarray(mse_test_array).size)[:]*it_print,mse_test_array[:],label="test")
plt.plot(np.arange(np.asarray(mse_train_array).size)[:]*it_print,mse_train_array[:],label="train")

# add legend
leg = ax.legend(loc=(0.4, 0.75),frameon=True,prop={'size': 80})
leg.get_frame().set_edgecolor('black')
leg.get_frame().set_linewidth(3)

plt.grid(True)
# add labels for axes
ax.set_xlabel("iterations")
plt.ylabel('MSE')



### Results: Forcast vs Actual

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 22))
plt.title("Forecast vs Actual", fontsize=100)
plt.plot(pd.Series(np.ravel(y_test)), "bo", markersize=40, label="Actual", color='green')
plt.plot(pd.Series(np.ravel(y_pred)), "r.", markersize=40, label="Forecast", color='red')

plt.legend(loc="lower left")
plt.xlabel(f'Time')
plt.ylabel(f'Energy [kWh]')
ax.grid(True)
plt.show()